In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [35]:
origin_df=pd.read_csv('https://production-bobsim.s3.ap-northeast-2.amazonaws.com/public_data/open_data_terrestrial_weather/origin/csv/2014-2020.csv', encoding='euc-kr')

In [36]:
origin_df.head()

,지점,지점명,일시,평균기온(°C),최저기온(°C),최고기온(°C),강수 계속시간(hr),일강수량(mm),최대 풍속(m/s),평균 풍속(m/s),최소 상대습도(pct),평균 상대습도(pct),합계 일조시간(hr),합계 일사량(MJ/m2)
0,90,속초,2014-01-01,6.6,4.1,8.9,NaN,NaN,9.8,5.4,22.0,31.3,7.0,NaN
1,90,속초,2014-01-02,4.0,-0.3,7.0,NaN,NaN,4.2,1.8,18.0,31.9,8.9,NaN
2,90,속초,2014-01-03,6.6,2.4,9.4,NaN,NaN,9.6,3.0,24.0,40.0,4.4,NaN
3,90,속초,2014-01-04,1.2,0.0,3.8,18.75,7.4,6.0,1.8,44.0,87.4,0.0,NaN
4,90,속초,2014-01-05,0.4,-2.0,2.7,0.83,0.0,5.1,1.7,78.0,90.0,3.7,NaN


### Null & Type check

In [37]:
origin_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214325 entries, 0 to 214324
Data columns (total 14 columns):
지점               214325 non-null int64
지점명              214325 non-null object
일시               214325 non-null object
평균기온(°C)         214086 non-null float64
최저기온(°C)         214300 non-null float64
최고기온(°C)         214305 non-null float64
강수 계속시간(hr)      22288 non-null float64
일강수량(mm)         77862 non-null float64
최대 풍속(m/s)       214217 non-null float64
평균 풍속(m/s)       214143 non-null float64
최소 상대습도(pct)     214246 non-null float64
평균 상대습도(pct)     213919 non-null float64
합계 일조시간(hr)      213966 non-null float64
합계 일사량(MJ/m2)    92353 non-null float64
dtypes: float64(11), int64(1), object(2)
memory usage: 22.9+ MB


In [38]:
origin_df[['지점', '합계 일사량(MJ/m2)']].groupby("지점")['합계 일사량(MJ/m2)'].count().value_counts().sum()

96

In [39]:
origin_df[['지점', '합계 일사량(MJ/m2)']].groupby("지점")['합계 일사량(MJ/m2)'].count().value_counts()

0       38
1        8
2272     8
2270     7
2271     5
2268     5
2        4
2266     3
2269     2
691      1
3        1
526      1
402      1
2199     1
2213     1
1270     1
1600     1
2249     1
1613     1
1268     1
2258     1
2267     1
2159     1
1265     1
2257     1
Name: 합계 일사량(MJ/m2), dtype: int64

### Null value %

In [40]:
origin_df.isnull().sum()/214325* 100 

지점                0.000000
지점명               0.000000
일시                0.000000
평균기온(°C)          0.111513
최저기온(°C)          0.011665
최고기온(°C)          0.009332
강수 계속시간(hr)      89.600840
일강수량(mm)         63.671060
최대 풍속(m/s)        0.050391
평균 풍속(m/s)        0.084918
최소 상대습도(pct)      0.036860
평균 상대습도(pct)      0.189432
합계 일조시간(hr)       0.167503
합계 일사량(MJ/m2)    56.909833
dtype: float64

## null handling
#### 강수 계속시간 , 일강수량, 합계일사량 -> 0
#### 나머지 -> mean()

In [41]:
origin_df[['일강수량(mm)','강수 계속시간(hr)']] = origin_df[['일강수량(mm)','강수 계속시간(hr)']].fillna(value=0)

In [42]:
origin_df['합계 일사량(MJ/m2)'] = origin_df['합계 일사량(MJ/m2)'].fillna(value=0)

In [ ]:
origin_df[['평균기온(°C)','최저기온(°C)','최고기온(°C)', '강수 계속시간(hr)', '일강수량(mm)', '최대 풍속(m/s)',        
          '평균 풍속(m/s)', '최소 상대습도(pct)', '평균 상대습도(pct)', '합계 일조시간(hr)']]=origin_df[['평균기온(°C)','최저기온(°C)','최고기온(°C)', '강수 계속시간(hr)', '일강수량(mm)', '최대 풍속(m/s)',        
          '평균 풍속(m/s)', '최소 상대습도(pct)', '평균 상대습도(pct)', '합계 일조시간(hr)']].fillna(origin_df.mean())



In [ ]:
origin_df.isnull().sum()

### 합계 일사량은 조사자체를 안하는 지역이 많아서 null value가 많다 ( 삭제 )

In [ ]:
origin_df.drop('합계 일사량(MJ/m2)', axis = 1)

** 숫자 피처들에 대한 데이터 분포 왜곡도 확인 후 높은 왜곡도를 가지는 피처 추출 **

In [ ]:
from scipy.stats import skew

features_index = origin_df.dtypes[origin_df.dtypes != 'object'].index

skew_features = origin_df[features_index].apply(lambda x : skew(x))

skew_features_top = skew_features[skew_features > 1]
print(skew_features_top.sort_values(ascending=False))
#print(skew_features.sort_values(ascending=False))

In [ ]:
origin_df.describe()

** 왜곡도가 1인 피처들은 로그 변환 적용 **

In [ ]:
import numpy as np
origin_df[skew_features_top.index] = np.log1p(origin_df[skew_features_top.index])

In [30]:
skew_features = origin_df[features_index].apply(lambda x : skew(x))
print(skew_features.sort_values(ascending=False))

강수 계속시간(hr)      3.536727
일강수량(mm)         2.196918
평균 풍속(m/s)       0.764090
합계 일사량(MJ/m2)    0.507175
최대 풍속(m/s)       0.332204
지점              -0.440484
최소 상대습도(pct)    -0.613731
합계 일조시간(hr)     -1.063731
평균 상대습도(pct)    -1.088218
평균기온(°C)              NaN
최저기온(°C)              NaN
최고기온(°C)              NaN
dtype: float64
